In [3]:

import scipy
import scipy.io as scio
import numpy as np
import os
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVC
clf = NuSVC(nu=0.3)



In [5]:
# t32 数据采样并保存

# 读取训练和测试数据
data_root = '/data/file/classification_data/T31/tmp/'
T31TrainData = scio.loadmat(os.path.join(data_root, 'train_sumX_2015_norm.mat'))['X']
T31TestData = scio.loadmat(os.path.join(data_root, 'test_sumX_2015_norm.mat'))['X']

T31TrainLabel = scio.loadmat(os.path.join(data_root, 'train_sumy_2015_label.mat'))['y']
T31TestLabel = scio.loadmat(os.path.join(data_root, 'test_sumy_2015_label.mat'))['y']


In [3]:
print(T31TrainData.shape)
print(T31TrainLabel.shape)
print(T31TestData.shape)
print(T31TestLabel.shape)

(5414, 448)
(5414, 1)
(64, 448)
(64, 1)


In [4]:
list = []

#   ------------------上采样------------------
from imblearn.over_sampling import RandomOverSampler 
ros = RandomOverSampler(random_state=42)
list.append(ros)

from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42)
list.append(sm)

from imblearn.over_sampling import BorderlineSMOTE 
blsm = BorderlineSMOTE(random_state=42)
list.append(blsm)

from imblearn.over_sampling import ADASYN 
ada = ADASYN(random_state=42)
list.append(ada)

# -----------------Combination of over- and under-sampling------------
from imblearn.combine import SMOTEENN 
sme = SMOTEENN(random_state=42)
list.append(sme)

from imblearn.combine import SMOTETomek 
smt = SMOTETomek(random_state=42)
list.append(smt)

# -----------------ubder sampling---------------------------
from imblearn.under_sampling import NearMiss 
nm = NearMiss()
list.append(nm)

from imblearn.under_sampling import TomekLinks 
tl = TomekLinks()
# list.append(tl)

from imblearn.under_sampling import RandomUnderSampler 
rus = RandomUnderSampler(random_state=42)
list.append(rus)

In [5]:
for method in list:
    print(str(method))

RandomOverSampler(random_state=42)
SMOTE(random_state=42)
BorderlineSMOTE(random_state=42)
ADASYN(random_state=42)
SMOTEENN(random_state=42)
SMOTETomek(random_state=42)
NearMiss()
RandomUnderSampler(random_state=42)


In [6]:
save_data_root = '/data/file/classification_data/T31/samplings/'
for i, x in enumerate(list) :
    print('-----------------开始----------------')
    current_dir = save_data_root + str(list[i])
    print(current_dir)
    print(str(list[i]))
    if not os.path.exists(current_dir):
        os.makedirs(current_dir)
    data_sampling, label_sampling = list[i].fit_resample(T31TrainData, T31TrainLabel)
    print(data_sampling.shape,  label_sampling.shape)
    clf.fit(data_sampling, label_sampling)
    print(clf.score(T31TestData, T31TestLabel))

    # 保存mat文件
    scipy.io.savemat(current_dir+"/data_sampling.mat", {'X': data_sampling})
    scipy.io.savemat(current_dir+"/label_sampling.mat", {'y': label_sampling})

-----------------开始----------------
/data/file/classification_data/T31/samplings/RandomOverSampler(random_state=42)
RandomOverSampler(random_state=42)
(10674, 448) (10674,)
0.625
-----------------开始----------------
/data/file/classification_data/T31/samplings/SMOTE(random_state=42)
SMOTE(random_state=42)
(10674, 448) (10674,)
0.578125
-----------------开始----------------
/data/file/classification_data/T31/samplings/BorderlineSMOTE(random_state=42)
BorderlineSMOTE(random_state=42)
(10674, 448) (10674,)
0.53125
-----------------开始----------------
/data/file/classification_data/T31/samplings/ADASYN(random_state=42)
ADASYN(random_state=42)
(10701, 448) (10701,)
0.578125
-----------------开始----------------
/data/file/classification_data/T31/samplings/SMOTEENN(random_state=42)
SMOTEENN(random_state=42)
(10041, 448) (10041,)
0.609375
-----------------开始----------------
/data/file/classification_data/T31/samplings/SMOTETomek(random_state=42)
SMOTETomek(random_state=42)
(10674, 448) (10674,)
0.5

In [6]:
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(T31TrainData, T31TrainLabel)
print(eec.score(T31TestData, T31TestLabel))

from imblearn.ensemble import BalancedBaggingClassifier 
bbc = BalancedBaggingClassifier(random_state=42)
bbc.fit(T31TrainData, T31TrainLabel)
print(bbc.score(T31TestData, T31TestLabel)) 

/home/roczhang/anaconda3/envs/dataAna/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
0.734375
/home/roczhang/anaconda3/envs/dataAna/lib/python3.6/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
0.671875
